In [ ]:
import joblib

In [ ]:
from keras.models import load_model
import numpy as np

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd

if not firebase_admin._apps:
    cred = credentials.Certificate("firebase_key.json")
    firebase_admin.initialize_app(cred)

db = firestore.client()


In [ ]:
def predict_next_days(model, scaler, data, n_steps=100, days=7):
    recent_data = data[-n_steps:]
    input_seq = scaler.transform(recent_data)
    predictions = []

    for _ in range(days):
        input_reshaped = input_seq.reshape(1, n_steps, -1)
        pred = model.predict(input_reshaped, verbose=0)
        predictions.append(pred[0][0])
        
        next_step = np.zeros((1, input_seq.shape[1]))  
        next_step[0][0] = pred
        input_seq = np.vstack((input_seq[1:], next_step)) 

    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1))


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email_report(predictions, to_email):
    msg = MIMEMultipart()
    msg['From'] = "your_email@gmail.com"
    msg['To'] = to_email
    msg['Subject'] = "GOOGL Stock Predictions Report"

    html = "<h3>Predictions for the next days</h3><ul>"
    for i, val in enumerate(predictions, 1):
        html += f"<li>Day {i}: {val[0]:.2f}</li>"
    html += "</ul>"

    msg.attach(MIMEText(html, 'html'))

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login("your_email@gmail.com", "your_app_password")
        server.send_message(msg)


In [ ]:
def forecast_and_send_email():
    scaler = joblib.load('scaler.gz')
    scaled_data = scaler.transform(db)


    model = load_model("your_trained_model.h5")


    predictions = predict_next_days(model, scaler, scaled_data, n_steps=100, days=7)

    send_email_report(predictions, "recipient_email@example.com")





In [ ]:
import schedule
import time
schedule.every().day.at("08:00").do(forecast_and_send_email)

while True:
    schedule.run_pending()
    time.sleep(60)  
